<a href="https://colab.research.google.com/github/Helena0811/DataScience_LOP/blob/master/Bag%20of%20words%20Meets%20Bags%20of%20Popcorn/Movie_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 초기 세팅

In [1]:
import pandas as pd
import numpy as np
import warnings
import os
import matplotlib.pyplot as plt
import seaborn as sns

warnings .filterwarnings('ignore')
SEED = 33

#데이터 로드

In [16]:
DATA = '/content/Bag of Words Meets Bags of Popcorn/Data'

#os.path.join으로 파일 가져옴
train = pd.read_csv(os.path.join(DATA, 'labeledTrainData.tsv'), delimiter='\t')
test = pd.read_csv(os.path.join(DATA, 'testData.tsv'), delimiter='\t')
unlabled_train = pd.read_csv(os.path.join(DATA, 'unlabeledTrainData.tsv'), delimiter='\t', error_bad_lines=False)

b'Skipping line 43043: expected 2 fields, saw 3\n'


In [6]:
print(train.shape)
train.head()

(25000, 3)


,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [17]:
print(test.shape)
test.head()

(25000, 2)


,id,review
0,12311_10,Naturally in a film who's main themes are of m...
1,8348_2,This movie is a disaster within a disaster fil...
2,5828_4,"All in all, this is a movie for kids. We saw i..."
3,7186_2,Afraid of the Dark left me with the impression...
4,12128_7,A very accurate depiction of small time mob li...


In [18]:
print(unlabled_train.shape)
unlabled_train.head()

(49998, 2)


,id,review
0,9999_0,"Watching Time Chasers, it obvious that it was ..."
1,45057_0,I saw this film about 20 years ago and remembe...
2,15561_0,"Minor Spoilers<br /><br />In New York, Joan Ba..."
3,7161_0,I went to see this film with a great deal of e...
4,43971_0,"Yes, I agree with everyone on this site this m..."


#전처리

In [39]:
#html 태그 제거
from bs4 import BeautifulSoup

#불용어 제거
#자주 쓰이지만 의미없는 단어들!
#해당 소스가 먹히지 않아 download 추가...
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [22]:
sample = train['review'][0]

In [20]:
sample

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

In [23]:
soup = BeautifulSoup(sample, 'html.parser')

In [25]:
#태그 제거됨
soup.text

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.The actual feature film bit when it finally starts is only on for 20 mi

In [28]:
#특수기호 제거
import re

#소문자 a-z, A-Z를 제외한 모든 문자는 공백으로 대체
cleaned = re.sub('[^a-zA-Z]', ' ', soup.text)

In [29]:
cleaned

'With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him The actual feature film bit when it finally starts is only on for    mi

In [30]:
#모두 소문자로 출력
cleaned.lower()

'with all this stuff going down at the moment with mj i ve started listening to his music  watching the odd documentary here and there  watched the wiz and watched moonwalker again  maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  some of it has subtle messages about mj s feeling towards the press and also the obvious message of drugs are bad m kay visually impressive but of course this is all about michael jackson so unless you remotely like mj in anyway then you are going to hate this and find it boring  some may call mj an egotist for consenting to the making of this movie but mj and most of his fans would say that he made it for the fans which if true is really nice of him the actual feature film bit when it finally starts is only on for    mi

In [54]:
#전처리 함수 만들기
def preprocessing(sentence):
  soup = BeautifulSoup(sample, 'html.parser')
  cleaned = re.sub('[^a-zA-Z]', ' ', soup.text)
  cleaned = cleaned.lower()
  cleaned = [word for word in cleaned.split() if word not in eng_stopwords]
  #현재 processing(sample)을 찍어보면 단어가 분리된 형태로 나오는데, 이를 다시 문장 형태로 만들기
  return ' '.join(cleaned)

In [32]:
s = ['This', 'is', 'a', 'pen']

In [40]:
eng_stopwords = stopwords.words('english')

In [ ]:
eng_stopwords

In [43]:
#불용어 사전에 포함된 단어 찾기
for word in s:
  if word in eng_stopwords:
    print(word)

is
a


In [ ]:
for word in sample.lower().split():
  if word in eng_stopwords:
    print(word)

#아까 만들던 전처리 함수로 가서 이를 적용해보자! Go to In [31]

In [55]:
preprocessing(sample)

'stuff going moment mj started listening music watching odd documentary watched wiz watched moonwalker maybe want get certain insight guy thought really cool eighties maybe make mind whether guilty innocent moonwalker part biography part feature film remember going see cinema originally released subtle messages mj feeling towards press also obvious message drugs bad kay visually impressive course michael jackson unless remotely like mj anyway going hate find boring may call mj egotist consenting making movie mj fans would say made fans true really nice actual feature film bit finally starts minutes excluding smooth criminal sequence joe pesci convincing psychopathic powerful drug lord wants mj dead bad beyond mj overheard plans nah joe pesci character ranted wanted people know supplying drugs etc dunno maybe hates mj music lots cool things like mj turning car robot whole speed demon sequence also director must patience saint came filming kiddy bad sequence usually directors hate workin

In [71]:
#각 데이터 파일에 있는 리뷰 모아서 처리하기!
all_review = pd.concat([train['review'], unlabled_train['review'], test['review']])

print(train['review'][0])
print(unlabled_train['review'][0])
print(test['review'][0])
all_review.head()

With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally star

0    With all this stuff going down at the moment w...
1    \The Classic War of the Worlds\" by Timothy Hi...
2    The film starts with a manager (Nicholas Bell)...
3    It must be assumed that those who praised this...
4    Superbly trashy and wondrously unpretentious 8...
Name: review, dtype: object

In [66]:
#전처리 한 상태의 데이터를 변수에 넣기 - 전처리 하고 저장하는데 오래걸리네ㅠㅠ
all_review_clean = all_review.apply(preprocessing)

In [70]:
#엥 왜 concat이 안된거같지...?
all_review_clean.head()

0    stuff going moment mj started listening music ...
1    stuff going moment mj started listening music ...
2    stuff going moment mj started listening music ...
3    stuff going moment mj started listening music ...
4    stuff going moment mj started listening music ...
Name: review, dtype: object

#CountVectorizer

In [72]:
#counter vectorizer
#단어에 대한 갯수를 세서 분석
from sklearn.feature_extraction.text import CountVectorizer

In [73]:
cv = CountVectorizer(analyzer='word', max_features=5000)

In [74]:
all_review_cv = cv.fit_transform(all_review_clean)

In [75]:
all_review_cv.shape

(99998, 166)